In [1]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "RedHatAI/Llama-4-Scout-17B-16E-Instruct-FP8-dynamic"
number_gpus = 2

sampling_params = SamplingParams(temperature=0.7, top_p=0.8, max_tokens=256)

tokenizer = AutoTokenizer.from_pretrained(model_id)

ModuleNotFoundError: No module named 'vllm'

In [2]:
import json
from pprint import pprint

import torch
import transformers
from environs import env
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    QuantoConfig,
    AutoProcessor,
    Llama4ForConditionalGeneration,
)

from local_funcs import chat_funcs, prompt_funcs
from yiutils.project_utils import find_project_root

proj_root = find_project_root("justfile")
data_dir = proj_root / "data"

print(transformers.__version__)
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

env.read_env(proj_root / ".env")
access_token = env("HUGGINGFACE_TOKEN")

path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists(), (
    f"Data file {path_to_mr_pubmed_data} does not exist."
)

with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)

article_data = mr_pubmed_data[0]

message_metadata = prompt_funcs.make_message_metadata(article_data["ab"])
message_results = prompt_funcs.make_message_results(article_data["ab"])


/home/b5a/ik18445.b5a/micromamba/envs/data-extraction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3
2.6.0
True
12.6


In [4]:
MODEL_ID = "RedHatAI/Llama-4-Scout-17B-16E-Instruct-FP8-dynamic"

device = "auto"
# dtype = torch.bfloat16
# dtype = torch.float8_e4m3fn

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    # torch_dtype=dtype,
    device_map=device,
    token=access_token,
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 23/23 [00:04<00:00,  4.68it/s]
Some weights of Llama4ForCausalLM were not initialized from the model checkpoint at RedHatAI/Llama-4-Scout-17B-16E-Instruct-FP8-dynamic and are newly initialized: ['model.layers.0.feed_forward.experts.down_proj', 'model.layers.0.feed_forward.experts.gate_up_proj', 'model.layers.1.feed_forward.experts.down_proj', 'model.layers.1.feed_forward.experts.gate_up_proj', 'model.layers.10.feed_forward.experts.down_proj', 'model.layers.10.feed_forward.experts.gate_up_proj', 'model.layers.11.feed_forward.experts.down_proj', 'model.layers.11.feed_forward.experts.gate_up_proj', 'model.layers.12.feed_forward.experts.down_proj', 'model.layers.12.feed_forward.experts.gate_up_proj', 'model.layers.13.feed_forward.experts.down_proj', 'model.layers.13.feed_forward.experts.gate_up_proj', 'model.layers.14.feed_forward.experts.down_proj', 'model.layers.14.feed_forward.experts.gate_up_proj', 'model.layers.15.feed_forward.experts.down_p

In [5]:
!nvidia-smi

Wed May  7 16:02:43 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GH200 120GB             On  | 00000009:01:00.0 Off |                    0 |
| N/A   27C    P0             137W / 900W |  88516MiB / 97871MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [6]:
model.config.torch_dtype

torch.float32

In [10]:
print(model)

Llama4ForCausalLM(
  (model): Llama4TextModel(
    (embed_tokens): Embedding(202048, 5120, padding_idx=200018)
    (layers): ModuleList(
      (0-2): 3 x Llama4TextDecoderLayer(
        (self_attn): Llama4TextAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (qk_norm): Llama4TextL2Norm(eps=1e-05)
        )
        (feed_forward): Llama4TextMoe(
          (experts): Llama4TextExperts(
            (act_fn): SiLU()
          )
          (router): Linear(in_features=5120, out_features=16, bias=False)
          (shared_expert): Llama4TextMLP(
            (gate_proj): Linear(in_features=5120, out_features=8192, bias=False)
            (up_proj): Linear(in_features=5120, out_features=8192, bias=False)
            (down_pro

In [7]:
messages = message_metadata
# messages = message_results
input_ids = tokenizer.apply_chat_template(
    conversation=messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)
print(input_ids.shape)
input_ids

torch.Size([1, 1034])


tensor([[200000, 200005,  15651,  ..., 140680, 200006,    368]],
       device='cuda:0')

In [9]:
terminators = [
    tokenizer.eos_token_id,
    # tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]
with torch.cuda.amp.autocast():
    outputs = model.generate(
        input_ids,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        # top_p=0.95,
    )
print(outputs.shape)
outputs

/local/user/1483800663/ipykernel_223406/3990874808.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.26 GiB. GPU 0 has a total capacity of 95.00 GiB of which 3.64 GiB is free. Including non-PyTorch memory, this process has 91.35 GiB memory in use. Of the allocated memory 89.58 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'You are a data scientist responsible for extracting accurate information from research papers. You answer each question with a single JSON string.<｜User｜>\n                This is an abstract from a Mendelian randomization study.\n                    "Alcohol consumption significantly impacts disease burden and has been linked to various diseases in observational studies. However, comprehensive meta-analyses using Mendelian randomization (MR) to examine drinking patterns are limited. We aimed to evaluate the health risks of alcohol use by integrating findings from MR studies. A thorough search was conducted for MR studies focused on alcohol exposure. We utilized two sets of instrumental variables-alcohol consumption and problematic alcohol use-and summary statistics from the FinnGen consortium R9 release to perform de novo MR analyses. Our meta-analysis encompassed 64 published and 151 de novo MR analyses across 76 distinct primary outcomes. Results show that a genetic predisposition 

In [11]:
response = outputs[0][input_ids.shape[-1] :]
result = tokenizer.decode(response, skip_special_tokens=True)
print(result)

Alright, so I need to figure out the exposures and outcomes from the given abstract. The abstract talks about alcohol consumption and problematic alcohol use as exposures. For the outcomes, it mentions several diseases like Parkinson's disease, prostate hyperplasia, rheumatoid arthritis, chronic pancreatitis, colorectal cancer, head and neck cancers, alcoholic liver disease, cirrhosis, pneumonia, and more.

Next, I have to categorize these. Alcohol consumption and problematic alcohol use are both behavioral, so they fall under the 'behavioural' category. The outcomes are various diseases, so I'll categorize each under the appropriate disease group. For example, Parkinson's disease is a nervous system disease, prostate hyperplasia is a neoplasm, and so on.

Now, looking at the methods used in the abstract, it mentions Mendelian randomization (MR) with two sets of instrumental variables and a meta-analysis. The methods listed in the example include things like two-sample MR, multivariabl